# Making Figures for Proposal

In [1]:
import lsdtopytools as lsd
from functions import *
import geopandas as gpd
import pandas as pd
import seaborn as sns
import rasterio
import rasterio.plot
from osgeo import gdal

/sciclone/home/ntlewis/research/.conda/lib/python3.8/site-packages/lsdtopytools/geoconvtools.py:79: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit(nopython = False)
/sciclone/home/ntlewis/research/.conda/lib/python3.8/site-packages/lsdtopytools/numba_tools.py:73: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit(nopython=False, parallel = True)


In [2]:
# NE bounds for quadrangles
pound_ap = (37.125,-82.5)
tazwell_south_vr = (37.125,-81.5)
montvale_br = (37.5,-79.625)
republican_grove_p = (37.000,-79.000)
bounds_list=[pound_ap,tazwell_south_vr,montvale_br,republican_grove_p]
color_list=['g','y','b','r']

In [3]:
ap_df, ap_dem = generate_ksn(north=pound_ap[0],east=pound_ap[1],name='pound_ap.tiff')
vr_df, vr_dem = generate_ksn(north=tazwell_south_vr[0],east=tazwell_south_vr[1],name='tazwell_south_vr.tiff')
br_df, br_dem = generate_ksn(north=montvale_br[0],east=montvale_br[1],name='montvale_br.tiff')
p_df, p_dem = generate_ksn(north=republican_grove_p[0],east=republican_grove_p[1],name='dillwyn_p.tiff')
df_list = [ap_df,vr_df,br_df,p_df] 

Loading the raster from file: /sciclone/home/ntlewis/research/working_files/data/pound_ap.tiff
LOADING TOOK 0.008354663848876953
I am recasting your nodata values to -9999 (standard LSDTT)
PREPROC TOOK 0.05178022384643555
Alright, let me summon control upon the c++ code ...
Got it.
INGESTINGINTO CPP TOOK 0.002271890640258789
I am an empty LSDJunctionNetwork
Nrows:1509 and ncols:1218
out Nrows:1509 and out ncols:1218

I am an empty LSDJunctionNetwork
Yes, yes, we know. Ignore the above depressing debugging text, that's perfectly normal to feel empty sometimes.
FINALISATION TOOK 8.916854858398438e-05
lsdtopytools is now ready to roll!
Carving: implementation of Lindsay (2016) DOI: 10.1002/hyp.10648
Filling: implementation of Wang and Liu (2006): https://doi.org/10.1080/13658810500433453
Processing...
I am going to carve/breach your depressions in order to force flow paths.
I am using an algorithm from Lindsay et al., 2016 DOI:https://doi.org/10.1002/hyp.10648
Implementation adapted from 

In [4]:
ninety_five = lambda df : np.percentile(df.m_chi,95)
ninety_five_dict = dict(zip(['ap','vr','br','p'],[ninety_five(x) for x in [ap_df,vr_df,br_df,p_df]]))
print(ninety_five_dict)

{'ap': 5.988968181610107, 'vr': 6.908539247512816, 'br': 6.526215147972107, 'p': 1.1773648262023926}


In [5]:
q2 = lambda df : df.elevation.describe()['50%']
q2_dict = dict(zip(['ap','vr','br','p'],[q2(x) for x in [ap_df,vr_df,br_df,p_df]]))
print(q2_dict)

{'ap': 548.8818359375, 'vr': 847.3070678710938, 'br': 351.63294982910156, 'p': 146.8021240234375}


In [6]:
dil = rasterio.open('data/dillwyn_p.tiff')

In [7]:
elevation=dil.read(1)[dil.read(1)>=0]
elevation

array([179.08583, 178.70387, 178.17395, ..., 161.1853 , 160.66853,
       159.94907], dtype=float32)

In [1]:
def make_fig1(df_list, _95_dict, q2_dict, color_list, bounds_list, ax):
    n=0
    for ax in ax.flat:
        key = list(_95_dict)[n]
        plot_name = key + 'plot'
        plot_name = sns.histplot(data=df_list[n][df_list[n].m_chi>_95_dict[key]], x='elevation', ax=ax, color=color_list[n], binwidth=10, shrink=0.8)
        total_stream_plot = sns.histplot(data=df_list[n], x='elevation', ax=ax, binwidth=10, color=color_list[n], alpha=0.3, shrink=0.8)
        ax.axvline(x=q2_dict[key], color='k', linestyle=':')
        ax.axvline(x=df_list[n][df_list[n].m_chi>_95_dict[key]].elevation.describe()['50%'], color=color_list[n], linestyle=':')
        n+=1

In [2]:
fig, ax = plt.subplots(2,2, figsize=(20,12), dpi=300, constrained_layout=True)
fig.suptitle('Elevation of Stream Reaches With $95^{th}$ Percentile $k_{sn}$ (Knickpoints) in Each Appalachian Province', fontsize=16)
sns.despine()
make_fig1(df_list=df_list, _95_dict=ninety_five_dict, q2_dict=q2_dict, color_list=color_list, bounds_list=bounds_list, ax=ax)
ax[0,0].set_title('Appalachian Plateau, Pound Quadrangle', fontsize=14)
ax[0,1].set_title('Valley and Ridge, Tazwell South Quadrangle', fontsize=14)
ax[1,0].set_title('Blue Ridge, Montvale Quadrangle', fontsize=14)
ax[1,1].set_title('Piedmont, Republican Grove Quadrangle', fontsize=14)
ax[0,0].text(x=750,y=1750,s=pound_ap, fontsize=14,c='g')
ax[0,1].text(x=1150,y=2000,s=tazwell_south_vr, fontsize=14,c='y')
ax[1,0].text(x=700,y=2250,s=montvale_br, fontsize=14,c='b')
ax[1,1].text(x=190,y=3500,s=republican_grove_p, fontsize=14,c='r')

NameError: name 'plt' is not defined